In [1]:
import cv2
import numpy as np

In [2]:
image1 = cv2.imread('right.jpg')
image2 = cv2.imread('left.jpg')

In [3]:
img1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

In [4]:
sift = cv2.SIFT_create()

In [5]:
keypoints1, descriptor1 = sift.detectAndCompute(img1_gray, None)
keypoints2, descriptor2 = sift.detectAndCompute(img2_gray, None)

In [6]:
matcher = cv2.BFMatcher()
matches =matcher.knnMatch(descriptor1, descriptor2, k=2)

In [7]:
#lowe's ratio!

good = []

for m, n  in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)

In [8]:
MIN_MATCH_COUNT = 10
if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good] ).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good] ).reshape(-1, 1, 2)
    
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    h, w, _ = image1.shape
    pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    
    image2 = cv2.polylines(image2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
    dst = cv2.warpPerspective(image1, M, (image2.shape[1] + image1.shape[1], image2.shape[0]))
    dst[0:image2.shape[0], 0:image2.shape[1]] = image2

    cv2.imshow("Stitched Image", dst)
    cv2.waitKey(0)
    cv2.destroyAllWindows()